### Github上でGoogleが管理しているリポジトリの情報をスクレイピングして下記の情報を取得する
* リポジトリ名
* 主要な言語
* スターの数
### 上記のデータを保存するためのDBを作成し，スクレイピングしたデータを保存する
* GithubのAPIは使わないこと（スクレイピングの課題なので）
* ちゃんと`time.sleep(1)`入れてね
### 保存したデータをSELECT文で表示する
### ソースコードはGithubフローに従って管理すること
1. ブランチ作成（add-exercise-1）
2. ステージング
3. コミット（コミットはキリのいいところでこまめにすること）
4. プッシュ
5. プルリク
6. コードレビュー，マージ（プルリクは自分で処理してよい）

In [18]:
# Webスクレイピングに最低限必要なライブラリをインポート
import requests
from bs4 import BeautifulSoup
import time
import sqlite3

In [19]:
# 初期設定
URL = "https://github.com" # アクセスするgithubのURL
repositories_url = "https://github.com/orgs/google/repositories"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}
response = requests.get(repositories_url, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")
time.sleep(1) # クローリングの際の負荷を削減するための待ち時間
empty_list = []
visited = set()

In [22]:
# リポジトリ名の抽出
for a in soup.find_all("a", href=True):
    href = a["href"]
    if href.startswith("/google/") and href.count("/") == 2:
        if href not in visited:
            visited.add(href)
            empty_list.append(a)

print(f"{len(empty_list)}")

print(empty_list)

# 主要な言語の抽出
language_list = []
for repo in empty_list:
    repo_url = URL + repo["href"]
    repo_response = requests.get(repo_url, headers=headers)
    repo_soup = BeautifulSoup(repo_response.text, "html.parser")
    time.sleep(1) # クローリングの際の負荷を削減するための待ち時間

    language_tag = repo_soup.find("span", attrs={"itemprop": "programmingLanguage"})
    if language_tag:
        language = language_tag.text.strip()
    else:
        language = "Not specified"
    language_list.append((repo.text.strip(), language))

print(language_list)

# スター数の抽出
star_list = []
for repo in empty_list:
    repo_url = URL + repo["href"]
    repo_response = requests.get(repo_url, headers=headers)
    repo_soup = BeautifulSoup(repo_response.text, "html.parser")
    time.sleep(1) # クローリングの際の負荷を削減するための待ち時間

    star_tag = repo_soup.find("a", class_="social-count js-social-count")
    if star_tag:
        stars = star_tag.text.strip()
    else:
        stars = "0"
    star_list.append((repo.text.strip(), stars))

print(star_list)

30
[<a class="Title-module__anchor--GmXUE Title-module__inline--oM0P7" href="/google/tunix"><span>tunix</span></a>, <a class="Title-module__anchor--GmXUE Title-module__inline--oM0P7" href="/google/skia"><span>skia</span></a>, <a class="Title-module__anchor--GmXUE Title-module__inline--oM0P7" href="/google/cel-java"><span>cel-java</span></a>, <a class="Title-module__anchor--GmXUE Title-module__inline--oM0P7" href="/google/meridian"><span>meridian</span></a>, <a class="Title-module__anchor--GmXUE Title-module__inline--oM0P7" href="/google/device-infra"><span>device-infra</span></a>, <a class="Title-module__anchor--GmXUE Title-module__inline--oM0P7" href="/google/ground-android"><span>ground-android</span></a>, <a class="Title-module__anchor--GmXUE Title-module__inline--oM0P7" href="/google/orbax"><span>orbax</span></a>, <a class="Title-module__anchor--GmXUE Title-module__inline--oM0P7" href="/google/zerocopy"><span>zerocopy</span></a>, <a class="Title-module__anchor--GmXUE Title-module__

In [30]:
path = ''
db_name = 'kadai.db'

try:
    # DB接続オブジェクトの作成
    conn = sqlite3.connect(path + db_name)

    # SQL(RDBを操作するための言語)を実行するためのmカーソルオブジェクトを取得
    cur = conn.cursor()

    # SQL文の作成
    # テーブルの作成
    sql = 'CREATE TABLE kadai (name TEXT, language TEXT, stars INT);'

    # SQL文の実行
    cur.execute(sql)
    print('テーブルを作成しました。')
    # テーブル名：kadai

except sqlite3.Error as e:
    print('エラーが発生しました:', e)

finally:
    # DBへの接続を閉じる
    conn.close()

エラーが発生しました: table kadai already exists


In [38]:
path = ''
db_name = 'car.db'

try:
    # DB接続オブジェクトの作成
    conn = sqlite3.connect(path + db_name)

    # SQL(RDBを操作するための言語)を実行するためのmカーソルオブジェクトを取得
    cur = conn.cursor()

    # SQL文の作成
    # データの挿入
    # INSERT INTO テーブル名　VALUES (列に対応したデータをカンマ区切りで);
    # 文字列データを挿入する場合は、文字列をシングルクォーテーションで挟む
    sql = "INSERT INTO kadai (name, language, stars) VALUES (?, ?, ?);"

    kadai = []
    for name, language, stars in zip(empty_list, language_list, star_list):

        # 名前の整形
        name_str = name.text.strip() if hasattr(name, "text") else str(name)

        # ★ タプル対策
        if isinstance(stars, tuple):
            stars = stars[0]
        stars_str = str(stars)
        stars_clean = stars_str.replace("★", "").replace(",", "").strip()
        stars_int = int(stars_clean) if stars_clean.isdigit() else 0

    kadai.append((name_str, language, stars_int))

    # SQL文の実行
    cur.executemany(sql, kadai)

    # 変更をDBに反映させる
    conn.commit()

except sqlite3.Error as e:
    print('エラーが発生しました:', e)

finally:
    # DBへの接続を閉じる
    conn.close()

エラーが発生しました: no such table: kadai


In [39]:
path = ''
db_name = 'car.db'

try:
    # DB接続オブジェクトの作成
    conn = sqlite3.connect(path + db_name)

    # SQL(RDBを操作するための言語)を実行するためのmカーソルオブジェクトを取得
    cur = conn.cursor()

    # データを参照するSQL
    # SELECT * FROM テーブル名;
    # * の部分は、取得したい列の名前を,区切りで指定することもできる
    sql = "SELECT * FROM kadai;"

    # SQL文の実行
    cur.execute(sql)

    # 変更をDBに反映させる
    # conn.commit()

except sqlite3.Error as e:
    print('エラーが発生しました:', e)

else:
    for row in cur:
        id, name, language, stars = row
        print(id, name, language, stars)

finally:
    # DBへの接続を閉じる
    conn.close()

エラーが発生しました: no such table: kadai
